In [2]:
import os
import sqlite3
import pandas as pd

# Paths
os.makedirs("data", exist_ok=True)
db_path  = "data/sap.db"        # << your requested SQLite file
csv_path = "data/sap500.csv"    # << your CSV

# Read CSV
df = pd.read_csv(csv_path)
df.columns = [c.strip().lower() for c in df.columns]      # normalize column names
df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m-%d")
df["symbol"] = "S&P500"  # index label (rename if you prefer)

# Create / open DB
conn = sqlite3.connect(db_path)

# Create table (once)
conn.execute("""
CREATE TABLE IF NOT EXISTS prices (
  symbol TEXT NOT NULL,
  date   TEXT NOT NULL,
  open   REAL,
  high   REAL,
  low    REAL,
  close  REAL,
  volume REAL,
  PRIMARY KEY (symbol, date)
);
""")

# Load data into table (replace for a clean run)
df[["symbol","date","open","high","low","close","volume"]].to_sql(
    "prices", conn, if_exists="replace", index=False
)

# Helpful indexes
conn.execute("CREATE INDEX IF NOT EXISTS idx_prices_symbol_date ON prices(symbol, date);")
conn.execute("CREATE INDEX IF NOT EXISTS idx_prices_date ON prices(date);")
conn.commit()

print("Database created at:", db_path)
print("Rows loaded:", len(df))


Database created at: data/sap.db
Rows loaded: 24540


In [3]:
import pandas as pd

# Coverage (rows + date range)
coverage = pd.read_sql_query("""
SELECT symbol, COUNT(*) AS rows, MIN(date) AS min_date, MAX(date) AS max_date
FROM prices
GROUP BY symbol;
""", conn)
display(coverage)

# Daily close series (for Excel / Tableau)
daily_close = pd.read_sql_query("""
SELECT date, close
FROM prices
WHERE symbol = 'S&P500'
ORDER BY date;
""", conn)
display(daily_close.head())

# Month-end close (uses window functions; if your SQLite is older, see fallback in next cell)
q_month_end = """
WITH tagged AS (
  SELECT
    date,
    substr(date,1,7) AS ym,
    close,
    ROW_NUMBER() OVER (PARTITION BY substr(date,1,7) ORDER BY date DESC) AS rn
  FROM prices
  WHERE symbol='S&P500'
)
SELECT ym AS month, date AS month_end_date, close AS month_end_close
FROM tagged
WHERE rn = 1
ORDER BY month;
"""
try:
    month_end = pd.read_sql_query(q_month_end, conn)
except Exception as e:
    print("SQLite lacks window functions; using pandas fallback.")
    tmp = daily_close.copy()
    tmp["ym"] = pd.to_datetime(tmp["date"]).dt.to_period("M")
    month_end = (tmp.sort_values("date")
                   .groupby("ym").tail(1)
                   .rename(columns={"date":"month_end_date","close":"month_end_close"}))
    month_end["month"] = month_end["ym"].astype(str)
    month_end = month_end[["month","month_end_date","month_end_close"]].reset_index(drop=True)

display(month_end.head())


,symbol,rows,min_date,max_date
0,S&P500,24540,1927-12-30,2025-09-11


,date,close
0,1927-12-30,17.660000
1,1928-01-03,17.760000
2,1928-01-04,17.719999
3,1928-01-05,17.549999
4,1928-01-06,17.660000


,month,month_end_date,month_end_close
0,1927-12,1927-12-30,17.660000
1,1928-01,1928-01-31,17.570000
2,1928-02,1928-02-29,17.260000
3,1928-03,1928-03-30,19.280001
4,1928-04,1928-04-30,19.750000


In [4]:
os.makedirs("exports", exist_ok=True)

daily_close.to_csv("exports/daily_close_sp500.csv", index=False)
month_end.to_csv("exports/month_end_close_sp500.csv", index=False)

print("Saved:")
print(" - exports/daily_close_sp500.csv")
print(" - exports/month_end_close_sp500.csv")

Saved:
 - exports/daily_close_sp500.csv
 - exports/month_end_close_sp500.csv
